<center><img src=https://raw.githubusercontent.com/feast-dev/feast/master/docs/assets/feast_logo.png width=400/></center>

# Deploying the Feature Store

### Introduction

Feast enables AI/ML teams to serve (and consume) features via feature stores. In this notebook, we will configure the feature stores and feature definitions, and deploy a Feast feature store server. We will also materialize (move) data from the offline store to the online store.

In Feast, offline stores support pulling large amounts of data for model training using tools like Redshift, Snowflake, Bigquery, and Spark. In contrast, the focus of Feast online stores is feature serving in support of model inference, using tools like Redis, Snowflake, PostgreSQL, and SQLite.

In this notebook, we will setup a file-based (Dask) offline store and SQLite online store. The online store will be made available through the Feast server.

This notebook assumes that you have prepared the data by running the notebook [01_Credit_Risk_Data_Prep.ipynb](01_Credit_Risk_Data_Prep.ipynb). 

### Setup

*The following code assumes that you have read the example README.md file, and that you have setup an environment where the code can be run. Please make sure you have addressed the prerequisite needs.*

In [ ]:
# For a clean start (if you have already run this notebook before)
# uncomment and run the following lines to purge the files
!rm Feature_Store/feature_store.yaml Feature_Store/Feature_Store/feature_definitions.py
!rm Feature_Store/data/registry.db Feature_Store/data/online_store.db

### Feast Feature Store Configuration

In [20]:
# Imports
import re
import time
import sqlite3
import subprocess
import datetime as dt
from feast import FeatureStore

For model training, we usually don't need (or want) a constantly running feature server. All we need is the ability to efficiently query and pull all of the training data at training time. In contrast, during model serving we need servers that are always ready to supply feature records in response to application requests. 

This training-serving dichotomy is reflected in Feast using "offline" and "online" stores. Offline stores are configured to work with database technologies typically used for training, while online stores are configured to use storage and streaming technologies that are popular for feature serving.

We need to create a `feature_store.yaml` config file to tell feast the structure we want in our offline and online feature stores. Below, we write the configuration for a local "Dask" offline store and local SQLite online store. We give the feature store a project name of `loan_applications`, and provider `local`. The registry is where the feature store will keep track of feature definitions and online store updates; we choose a file location in this case.

See the [feature_store.yaml](https://docs.feast.dev/reference/feature-repository/feature-store-yaml) documentation for further details. 

In [21]:
%%writefile Feature_Store/feature_store.yaml

project: loan_applications
registry: data/registry.db
provider: local
offline_store:
    type: dask
online_store:
    type: sqlite
    path: data/online_store.db
entity_key_serialization_version: 2

Writing Feature_Store/feature_store.yaml


### Feature Definitions

We also need to create feature definitions and other feature constructs in a python file, which we name `feature_definitions.py`. For our purposes, we define the following:

- Data Source: connections to data storage or data-producing endpoints
- Entity: primary key fields which can be used for joining data
- FeatureView: collections of features from a data source
- FeatureService: collection of FeatureViews

For more information on these, see the [Concepts](https://docs.feast.dev/getting-started/concepts) section of the Feast documentation.

In [22]:
%%writefile Feature_Store/feature_definitions.py

# Imports
import os
from pathlib import Path
from feast import (
    FileSource,
    Entity,
    FeatureView,
    Field,
    FeatureService
)
from feast.types import Float32
from feast.data_format import ParquetFormat

CURRENT_DIR = os.path.abspath(os.curdir)

# Data Sources
# A data source tells Feast where the data lives
data_a = FileSource(
    file_format=ParquetFormat(),
    path=Path(CURRENT_DIR,"data/train_a.parquet").as_uri()
)
data_b = FileSource(
    file_format=ParquetFormat(),
    path=Path(CURRENT_DIR,"data/train_b.parquet").as_uri()
)
data_test = FileSource(
    file_format=ParquetFormat(),
    path=Path(CURRENT_DIR,"data/test.parquet").as_uri()
)   

# Entity
# An entity tells Feast the column it can use to join tables
loan_id = Entity(
    name = "loan_id",
    join_keys = ["ID"]
)

# Feature views
# A feature view is how Feast groups features
features_a = FeatureView(
    name="data_a",
    entities=[loan_id],
    schema=[
        Field(name="duration", dtype=Float32),
        Field(name="credit_amount",dtype=Float32),
        Field(name="installment_commitment",dtype=Float32)
    ],
    source=data_a
)
features_b = FeatureView(
    name="data_b",
    entities=[loan_id],
    schema=[
        Field(name="residence_since",dtype=Float32),
        Field(name="age",dtype=Float32),
        Field(name="existing_credits",dtype=Float32),
        Field(name="num_dependents",dtype=Float32)
    ],
    source=data_b
)
features_test = FeatureView(
    name="data_test",
    entities=[loan_id],
    schema=[
        Field(name="duration", dtype=Float32),
        Field(name="credit_amount",dtype=Float32),
        Field(name="installment_commitment",dtype=Float32),
        Field(name="residence_since",dtype=Float32),
        Field(name="age",dtype=Float32),
        Field(name="existing_credits",dtype=Float32),
        Field(name="num_dependents",dtype=Float32)
    ],
    source=data_test    
)

# Feature Service
# a feature service in Feast represents a logical group of features
loan_fs = FeatureService(
    name="loan_fs",
    features=[features_a, features_b]
)

Overwriting Feature_Store/feature_definitions.py


### Applying the Configuration and Definitions

Now that we have our feature store configuration (`feature_store.yaml`) and feature definitions (`feature_definitions.py`), we are ready to "apply" them. The `feast apply` command creates a registry file (`Feature_Store/data/registry.db`) and sets up data connections; in this case, it creates a SQLite database (`Feature_Store/data/online_store.db`).

In [23]:
# Run 'feast apply' in the Feature_Store directory
!feast --chdir ./Feature_Store apply

10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
Created entity loan_id
Created feature view data_b
Created feature view data_a
Created feature view data_test
Created feature service loan_fs

10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
10/14/2024 09:03:29 PM root WARNING: Cannot use sqlite_vec for vector search
Created sqlite table loan_applications_data_a
Created sqlite table loan_applications_data_b
Created sqlite table loan_applications_data_test



In [24]:
# List the Feature_Store/data/ directory to see newly created files
!ls -nlh Feature_Store/data/

total 248
-rw-r--r--  1 501  20    40K Oct 14 21:03 online_store.db
-rw-r--r--  1 501  20   3.3K Oct 14 21:03 registry.db
-rw-r--r--  1 501  20    12K Oct 14 10:04 test.parquet
-rw-r--r--  1 501  20   5.8K Oct 14 10:04 test_y.parquet
-rw-r--r--  1 501  20    21K Oct 14 10:04 train_a.parquet
-rw-r--r--  1 501  20    18K Oct 14 10:04 train_b.parquet
-rw-r--r--  1 501  20    14K Oct 14 10:04 train_y.parquet


Note that while `feast apply` set up the `sqlite` online database, `online_store.db`, no data has been added to the online database as of yet. We can verify this by connecting with the `sqlite3` library.

In [25]:
# Connect to sqlite database
conn = sqlite3.connect("Feature_Store/data/online_store.db")
cursor = conn.cursor()
# Query table data (3 tables)
print(
    "Online Store Tables:           ",
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
)
print(
    "loan_applications_data_a data: ",
    cursor.execute("SELECT * FROM loan_applications_data_a").fetchall()
)
print(
    "loan_applications_data_b data: ",
    cursor.execute("SELECT * FROM loan_applications_data_b").fetchall()
)
conn.close()

Online Store Tables:            [('loan_applications_data_a',), ('loan_applications_data_b',), ('loan_applications_data_test',)]
loan_applications_data_a data:  []
loan_applications_data_b data:  []


Since we have used `feast apply` to create the registry, we can now use the Feast Python SDK to interact with our new feature store. To see other possible commands see the [Feast Python SDK documentation](https://rtd.feast.dev/en/master/).

In [26]:
# Get feature store config
store = FeatureStore(repo_path="./Feature_Store")
store.config

RepoConfig(project='loan_applications', provider='local', registry_config='data/registry.db', online_config={'type': 'sqlite', 'path': 'data/online_store.db'}, offline_config={'type': 'dask'}, batch_engine_config='local', feature_server=None, flags=None, repo_path=PosixPath('Feature_Store'), entity_key_serialization_version=2, coerce_tz_aware=True)

In [27]:
# List feature views
feature_views = store.list_batch_feature_views()
for fv in feature_views:
    print(f"Feature view: {fv.name}  |  Features: {fv.features}")

Feature view: data_b  |  Features: [residence_since-Float32, age-Float32, existing_credits-Float32, num_dependents-Float32]
Feature view: data_a  |  Features: [duration-Float32, credit_amount-Float32, installment_commitment-Float32]
Feature view: data_test  |  Features: [duration-Float32, credit_amount-Float32, installment_commitment-Float32, residence_since-Float32, age-Float32, existing_credits-Float32, num_dependents-Float32]


### Deploying the Feature Store Server

If you wish to share a feature store with your team, Feast provides feature servers. To spin up an offline feature server process, we can use the `feast serve_offline` command, while to spin up a Feast online feature server, we use the `feast serve` command.

Let's spin up an offline and an online server that we can use in the subsequent notebooks to get features during model training and model serving.

In [28]:
# Feast offline server process
offline_server_proc = subprocess.Popen(["feast", "--chdir", "./Feature_Store", "serve_offline"])

In [29]:
# Feast online server (master and worker) processes
online_server_proc = subprocess.Popen(["feast", "--chdir", "./Feature_Store", "serve"])
time.sleep(5)

/Users/ddowler/Code/Feast/feast/examples/credit-risk-end-to-end/venv-py3.11/lib/python3.11/site-packages/uvicorn/workers.py:16: DeprecationWarning: The `uvicorn.workers` module is deprecated. Please use `uvicorn-worker` package instead.
For more details, see https://github.com/Kludex/uvicorn-worker.
  warnings.warn(
[2024-10-14 21:03:30 -0600] [95955] [INFO] Starting gunicorn 23.0.0
[2024-10-14 21:03:30 -0600] [95955] [INFO] Listening at: http://127.0.0.1:6566 (95955)
[2024-10-14 21:03:30 -0600] [95955] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2024-10-14 21:03:30 -0600] [95962] [INFO] Booting worker with pid: 95962
[2024-10-14 21:03:30 -0600] [95962] [INFO] Started server process [95962]
[2024-10-14 21:03:30 -0600] [95962] [INFO] Waiting for application startup.
[2024-10-14 21:03:30 -0600] [95962] [INFO] Application startup complete.


Note that the output helpfully let's us know that the online server is "Listening at: http://127.0.0.1:6566". 

In [30]:
# Check the Feast server PIDs
print("Online Feast Server PID:  ", online_server_proc.pid)
print("Offline Feast Server PID: ", offline_server_proc.pid)

Online Feast Server PID:   95955
Offline Feast Server PID:  95954


In [31]:
# List running Feast processes (paths redacted)
running_procs = !ps -ef | grep feast | grep serve

for line in running_procs:
    redacted = re.sub(r'/*[^\s]*(?P<cmd>(python )|(feast ))', r'**/\g<cmd>', line)
    print(redacted)

  501 95954 84045   0  9:03PM ??         0:02.59 **/python **/feast --chdir ./Feature_Store serve_offline
  501 95955 84045   0  9:03PM ??         0:02.67 **/python **/feast --chdir ./Feature_Store serve
  501 95962 95955   0  9:03PM ??         0:00.03 **/python **/feast --chdir ./Feature_Store serve
  501 95988 84045   0  9:03PM ??         0:00.01 /bin/zsh -c ps -ef | grep **/feast | grep serve


Let's use a curl command to test a query on the online server.

In [32]:
# Online server query
cmd = """http://localhost:6566/get-online-features \
    -d '{ 
            "feature_service": "loan_fs",
            "entities": {"ID": [18, 764]}
        }'
"""

response = !curl -X POST {cmd}

127.0.0.1:52399 - "POST /get-online-features HTTP/1.1" 200


10/14/2024 09:03:34 PM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
10/14/2024 09:03:34 PM root WARNING: Cannot use sqlite_vec for vector search


In [33]:
response

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '100  1266  100  1170  100    96   110k   9251 --:--:-- --:--:-- --:--:--  123k',
 '{"metadata":{"feature_names":["ID","credit_amount","installment_commitment","duration","num_dependents","existing_credits","residence_since","age"]},"results":[{"values":[18,764],"statuses":["PRESENT","PRESENT"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[null,null],"statuses":["NOT_FOUND","NOT_FOUND"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[null,null],"statuses":["NOT_FOUND","NOT_FOUND"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[null,null],"statuses":["NOT_FOUND","NOT_FOUND"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},

### Materialize Features to the Online Store

Let's use the SDK feature store object (that we set up above) to "materialize" data; this is Feast lingo for moving/updating data from the offline store to the online store.

In [34]:
# Materialize
# Recall that we mocked the outcome data to have timestamps from 
# 'Tue Sep 24 12:00:00 2023'out to "Wed Oct  9 12:00:00 2023"
# The loan outcome timestamps were then lagged by 30-90 days (which is Jan 7 12:00:00 2024)
res = store.materialize(
    start_date=dt.datetime(2023,9,24,12,0,0),
    end_date=dt.datetime(2024,1,7,12,0,0)
)

Materializing 3 feature views from 2023-09-24 12:00:00-06:00 to 2024-01-07 12:00:00-07:00 into the sqlite online store.

data_b:


100%|██████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 32269.77it/s]


data_a:


100%|██████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 42909.03it/s]


data_test:


100%|██████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 19657.42it/s]


Now, we can query the SQLite database again and see that we get data in the response! Unfortunately, the data is stored in binary strings.

In [35]:
# Query the online store database to verify materialized data
conn = sqlite3.connect("Feature_Store/data/online_store.db")
cursor = conn.cursor()
print(
    "loan_applications_data_a data: ",
    cursor.execute("SELECT * FROM loan_applications_data_a LIMIT 2").fetchall()
)
print(
    "loan_applications_data_b data: ",
    cursor.execute("SELECT * FROM loan_applications_data_b LIMIT 2").fetchall()
)
conn.close()

loan_applications_data_a data:  [(b'\x02\x00\x00\x00ID\x04\x00\x00\x00\x08\x00\x00\x00Z\x00\x00\x00\x00\x00\x00\x00', 'duration', b'5\x00\x00@A', None, '2023-09-24 12:09:16', None), (b'\x02\x00\x00\x00ID\x04\x00\x00\x00\x08\x00\x00\x00Z\x00\x00\x00\x00\x00\x00\x00', 'credit_amount', b'5\x00\x80\x1aD', None, '2023-09-24 12:09:16', None)]
loan_applications_data_b data:  [(b'\x02\x00\x00\x00ID\x04\x00\x00\x00\x08\x00\x00\x00Z\x00\x00\x00\x00\x00\x00\x00', 'residence_since', b'5\x00\x00\x80@', None, '2023-09-24 12:09:16', None), (b'\x02\x00\x00\x00ID\x04\x00\x00\x00\x08\x00\x00\x00Z\x00\x00\x00\x00\x00\x00\x00', 'age', b'5\x00\x00`B', None, '2023-09-24 12:09:16', None)]


If we use the `get-online-features` REST API command, we get a readable JSON response.

In [36]:
# curl command to online server to get data from the online store
cmd = """http://localhost:6566/get-online-features \
    -d '{ 
            "feature_service": "loan_fs",
            "entities": {"ID": [18, 764]}
        }'
"""

response = !curl -X POST {cmd}

127.0.0.1:52401 - "POST /get-online-features HTTP/1.1" 200


10/14/2024 09:03:34 PM root WARNING: _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.


In [37]:
response

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '100  1235  100  1139  100    96   392k  33874 --:--:-- --:--:-- --:--:--  603k',
 '{"metadata":{"feature_names":["ID","credit_amount","installment_commitment","duration","num_dependents","existing_credits","residence_since","age"]},"results":[{"values":[18,764],"statuses":["PRESENT","PRESENT"],"event_timestamps":["1970-01-01T00:00:00Z","1970-01-01T00:00:00Z"]},{"values":[12579.0,2463.0],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2023-10-04T11:18:21Z","2023-10-05T07:35:12Z"]},{"values":[4.0,4.0],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2023-10-04T11:18:21Z","2023-10-05T07:35:12Z"]},{"values":[24.0,24.0],"statuses":["PRESENT","PRESENT"],"event_timestamps":["2023-10-04T11:18:21Z","2023-10-05T07:35:12Z"]},{"values"

### Server Shutdown

The Feast offline and online servers are designed to run continuously. The training and serving notebooks will assume that they are running, so **DO NOT RUN THE FOLLOWING CODE UNTIL YOU ARE DONE EXPERIMENTING**. 

Only after you are done with the other notebooks, you can run the below cells to shutdown the servers. Enter "y" when prompted to shutdown the servers.

In [17]:
# Cleanup: terminate feast server processes
answer = input("Are you sure you are ready to SHUTDOWN? [y|n]")
if answer.strip().lower() in ("y", "yes"):
    online_server_proc.terminate()
    offline_server_proc.terminate()
    time.sleep(3)
else:
    print("Servers NOT shut down")

Are you sure you are ready to SHUTDOWN? [y|n] y


[2024-10-14 21:00:30 -0600] [84242] [INFO] Handling signal: term
[2024-10-14 21:00:30 -0600] [84244] [INFO] Shutting down
[2024-10-14 21:00:30 -0600] [84244] [INFO] Error while closing socket [Errno 9] Bad file descriptor
[2024-10-14 21:00:30 -0600] [84244] [INFO] Waiting for application shutdown.
[2024-10-14 21:00:30 -0600] [84244] [INFO] Application shutdown complete.
[2024-10-14 21:00:30 -0600] [84244] [INFO] Finished server process [84244]
[2024-10-14 21:00:30 -0600] [84242] [ERROR] Worker (pid:84244) was sent SIGTERM!
[2024-10-14 21:00:30 -0600] [84242] [INFO] Shutting down: Master


In [18]:
# Show running commands with "feast" and "serve" in the name (but not "grep")
# (Since the servers were shut down, you shouldn't see any output)
!ps -ef | grep feast | grep serve | grep -v grep